**Classes:**
*Abuse*, arrest, *assault*, burglary, fighting, road accidents, shooting, shoplifting, vandalism, normal.

Out of: **abuse**, **arrest**, arson, **assault**, burglary, **fighting**, **roadaccidents**, **robbery**, **shooting**, shoplifting, **stealing**, **vandalism**, **normal**

**Train-test split:**

No. of videos in train dataset: 224

No. of normal videos in train dataset: 123

Ratio: 0.549

No. of videos in test dataset: 47

No. of normal videos in test dataset: 26

Ratio: 0.553

**Train-test split = 80:20**


# Load anomaly class folder

In [ ]:
# @title Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Unzip uploaded folder in Colab
!unzip "/content/drive/MyDrive/anomaly_detection/Test_dataset_2.zip" -d "/content/Test_dataset_2_unzipped"

In [ ]:
pip install pytorchyolo

# Run YOLO

In [ ]:
# @title Import dependencies
import cv2
from pytorchyolo import detect, models
import torch
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
import numpy as np
from PIL import Image

In [ ]:
cv2.__version__

'4.8.0'

# Run Deep SORT

In [ ]:
!git clone https://github.com/abhyantrika/nanonets_object_tracking/

Cloning into 'nanonets_object_tracking'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 125 (delta 21), reused 11 (delta 11), pack-reused 79
Receiving objects: 100% (125/125), 13.59 MiB | 17.70 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/anomaly_detection/nanonets_object_tracking-master')

In [ ]:
!ls "/content/drive/MyDrive/anomaly_detection/nanonets_object_tracking-master"

ckpts	     det	    README.md	      siamese_dataloader.py  siamese_train.py
deep_sort    get_images.py  requirements.txt  siamese_net.py	     test_on_video.py
deepsort.py  __pycache__    roi.jpg	      siamese_test.py


In [ ]:
import torchvision.utils
import numpy as np
import cv2
import csv
from deepsort import *

In [ ]:
def get_gt(image,frame_id,gt_dict):
	# print("IN GET_GT")
	if frame_id not in gt_dict.keys() or gt_dict[frame_id]==[]:
		return None,None

	frame_info = gt_dict[frame_id]

	detections = []
	ids = []
	out_scores = []
	for i in range(len(frame_info)):

		coords = frame_info[i]['coords']

		x1,y1,x2,y2 = coords
		w = x2 - x1
		h = y2 - y1
		# x2 = x1 + w
		# y2 = y1 + h

		# xmin = min(x1,x2)
		# xmax = max(x1,x2)
		# ymin = min(y1,y2)
		# ymax = max(y1,y2)

		detections.append([x1,y1,w,h])
		out_scores.append(frame_info[i]['conf'])

	return detections,out_scores


def get_dict(filename):
	with open(filename) as f:
		d = f.readlines()

	d = list(map(lambda x:x.strip(),d))

	last_frame = int(d[-1].split(',')[0])

	gt_dict = {x:[] for x in range(last_frame+1)}

	for i in range(len(d)):
		a = list(d[i].split(','))
		a = list(map(float,a))

		coords = a[2:6]
		confidence = a[6]
		gt_dict[a[0]].append({'coords':coords,'conf':confidence})

	return gt_dict

def get_mask(filename):
	mask = cv2.imread(filename,0)
	mask = mask / 255.0
	return mask

In [ ]:
anomaly_map = {'Abuse':0,
               'Arrest':1,
               'Assault':2,
               'Burglary':3,
               'Fighting':4,
               'Normal':5,
               'RoadAccidents':6,
               'Shooting':7,
               'Shoplifting':8,
               'Vandalism':9}

In [ ]:
def annotations(vid_file_name):
  # anno_file = open('/content/drive/MyDrive/anomaly_detection/Temporal_Anomaly_Annotation_for_Testing_Videos.txt', 'r')
  # annotations = anno_file.read()
  # annotations = annotations.split("\n")
  # for i in range(291):
  #   annotations[i] = annotations[i].split('  ')
  # for annotation in annotations:
  #   # print(annotation)
  #   if annotation[0] == vid_file_name:
  #     anomaly_class = anomaly_map[annotation[1]]
  #     result = [int(annotation[2]), int(annotation[3]), anomaly_class] # Returns result: [anomaly_start_frame, anomaly_end_frame, anomaly_type]
  # anno_file.close()
  result = []

  classes_dict = {}
  classes_file_name = "/content/drive/MyDrive/anomaly_detection/det/classes_{}.txt".format(vid_file_name)
  classes_file = open(classes_file_name, 'r')
  classes = classes_file.readlines()
  classes_file.close()
  # classes = classes.split("\n")
  # Need to get obj class from classes file
  for frame_class in classes:
    frame_class = frame_class.split(',')
    classes_dict[int(frame_class[0])] = float(frame_class[1])

  return result, classes_dict

# result, classes_dict = annotations("Abuse028_x264.mp4")
# result
# classes_dict

In [ ]:
import re

def extract_anomaly_type(filename):
  """
  Extracts the anomaly type from a filename.

  Args:
    filename: The filename to extract the anomaly type from.

  Returns:
    The anomaly type, or None if not found.
  """

  match = re.search(r'^(\D+)', filename)
  if match:
    anomaly_type = match.group(1)
    if anomaly_type == 'Normal_Videos_':
      return anomaly_map['Normal']
    else:
      return anomaly_map[anomaly_type]
  else:
    return None

# filename = 'Normal_Videos_910_x264.mp4'
# anomaly_type = extract_anomaly_type(filename)
# print(anomaly_type)

5


In [ ]:
# @title Run YOLO and DeepSORT to generate detection files for each video

import os

# CONFIG_PATH = "/Users/deepan_roy/Documents/2-2/SOP/PyTorch-YOLOv3-master/config/yolov3.cfg"
# WEIGHTS_PATH = "/Users/deepan_roy/Documents/2-2/SOP/PyTorch-YOLOv3-master/yolov3.weights"
CONFIG_PATH = "/content/drive/MyDrive/anomaly_detection/YOLOv3/yolov3.cfg"
WEIGHTS_PATH = "/content/drive/MyDrive/anomaly_detection/YOLOv3/yolov3.weights"

# Load the YOLO model
model = models.load_model(
  CONFIG_PATH,
  WEIGHTS_PATH)

# vid_name = "Abuse028_x264.mp4"
PATH = '/content/Test_dataset_2_unzipped/Test_dataset_2/'
for vid_name in l2:
  vidfile = PATH + vid_name
  print(vidfile + '...')
  cap = cv2.VideoCapture(vidfile)
  fps = cap.get(cv2.CAP_PROP_FPS)
  print("Number of frames: {}".format(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

  pred_cnt = 0
  detections = ""
  classes = ""
  while cap.isOpened():
      is_read, frame = cap.read()
      if is_read:
          pos_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
          boxes = detect.detect_image(model, frame)
          boxes = np.asarray(boxes)
          for box in boxes:
                detections += "{},-1,{},{},{},{},{},-1,-1,-1\n".format(pos_frame, box[0], box[1], box[2], box[3], box[4])
                classes += "{},{}\n".format(pos_frame, box[5])
          # print("{}... ".format(pos_frame))
      else:
          # cap.set(cv2.CAP_PROP_POS_FRAMES, pos_frame-1)
          cv2.waitKey(1000)
          print("Error")
      if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
          # If the number of captured frames is equal to the total number of frames, we stop
          break
      if cv2.waitKey(25) & 0xFF == ord("q"):
          break
  cap.release()
  cv2.destroyAllWindows()
  # print(detections)

  det_path = "/content/drive/MyDrive/anomaly_detection/det" + "/det_{}.txt".format(vid_name)
  with open(det_path, 'w') as det_file:
      det_file.write(detections)
  classes_path = "/content/drive/MyDrive/anomaly_detection/det" + "/classes_{}.txt".format(vid_name)
  with open(classes_path, 'w') as classes_file:
      classes_file.write(classes)



  # ========== DEEPSORT ==============




  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Select device for inference

  #Load detections for the video. Options available: yolo,ssd and mask-rcnn
  # vid_name = "Abuse028_x264.mp4"
  det_file_name = "/content/drive/MyDrive/anomaly_detection/det/det_{}.txt".format(vid_name)
  gt_dict = get_dict(det_file_name)
  # print(gt_dict)

  cap = cv2.VideoCapture(PATH + vid_name)

  #Initialize deep sort.
  deepsort = deepsort_rbc(wt_path='/content/drive/MyDrive/anomaly_detection/nanonets_object_tracking-master/ckpts/model640.pt')

  # deepsort.metric = deepsort.metric.to(device)
  # deepsort.tracker = deepsort.tracker.to(device)
  deepsort.gaussian_mask = deepsort.gaussian_mask.to(device)
  # deepsort.transforms = deepsort.transforms.to(device)
  # print(deepsort.metric.is_cuda)
  # print(deepsort.tracker.is_cuda)
  # print(deepsort.gaussian_mask.is_cuda)
  # print(deepsort.transforms.is_cuda)

  frame_id = 1


  anomaly_info, classes = annotations(vid_name)

  inputs = []

  while True:
    print(frame_id)

    ret,frame = cap.read()
    if ret is False:
      frame_id+=1
      break

    # frame = frame * mask
    frame = frame.astype(np.uint8)
    detections,out_scores = get_gt(frame,frame_id,gt_dict)

    if detections is None:
      print("No dets")
      frame_id+=1
      continue

    detections = np.array(detections)
    out_scores = np.array(out_scores)
    # print(len(detections) == 0)
    tracker,detections_class = deepsort.run_deep_sort(frame,out_scores,detections)

    for track in tracker.tracks:
      if not track.is_confirmed() or track.time_since_update > 1:
        continue

      bbox = track.to_tlbr() #Get the corrected/predicted bounding box
      id_num = str(track.track_id) #Get the ID for the particular track.
      features = track.features #Get the feature vector corresponding to the detection.

      pos_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

      # Read class for the frame in the current file from classes file
      obj_class = classes[pos_frame] if pos_frame in classes else -1
      # Read anomaly type, anomalous frame numbers from annotations file for the current file
      # anomaly_in_frame = anomaly_info[2] if pos_frame in range(anomaly_info[0], (anomaly_info[1] + 1)) else 5
      anomaly_in_frame = extract_anomaly_type(vid_name)

      input = [pos_frame, track.track_id, bbox[0], bbox[1], bbox[2], bbox[3], obj_class, anomaly_in_frame]	# How to add class and anomaly?
      inputs.append(input)

    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

    frame_id+=1

  csv_file_name = '/content/drive/MyDrive/anomaly_detection/Test/test_{}.csv'.format(vid_name)
  fields = ['frame_num', 'track_id', 'x_min', 'y_min', 'x_max', 'y_max', 'object_class', 'anomaly_in_frame']

  with open(csv_file_name, 'w') as csvfile:
      # creating a csv writer object
      csvwriter = csv.writer(csvfile)

      # writing the fields
      csvwriter.writerow(fields)

      # writing the data rows
      csvwriter.writerows(inputs)
  print('test_{}.csv done'.format(vid_name))

from google.colab import runtime
runtime.unassign()

In [ ]:
# # @title Run Deep SORT to generate input CSV files for LSTM

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Select device for inference

# #Load detections for the video. Options available: yolo,ssd and mask-rcnn
# vid_name = "Abuse028_x264.mp4"
# det_file_name = "/content/drive/MyDrive/anomaly_detection/det/det_{}.txt".format(vid_name)
# gt_dict = get_dict(det_file_name)
# # print(gt_dict)

# cap = cv2.VideoCapture('/content/drive/MyDrive/anomaly_detection/{}'.format(vid_name))

# #Initialize deep sort.
# deepsort = deepsort_rbc(wt_path='/content/drive/MyDrive/anomaly_detection/nanonets_object_tracking-master/ckpts/model640.pt')

# # deepsort.metric = deepsort.metric.to(device)
# # deepsort.tracker = deepsort.tracker.to(device)
# deepsort.gaussian_mask = deepsort.gaussian_mask.to(device)
# # deepsort.transforms = deepsort.transforms.to(device)
# # print(deepsort.metric.is_cuda)
# # print(deepsort.tracker.is_cuda)
# print(deepsort.gaussian_mask.is_cuda)
# # print(deepsort.transforms.is_cuda)

# frame_id = 1


# anomaly_info, classes = annotations(vid_name)

# inputs = []

# while True:
# 	print(frame_id)

# 	ret,frame = cap.read()
# 	if ret is False:
# 		frame_id+=1
# 		break

# 	# frame = frame * mask
# 	frame = frame.astype(np.uint8)
# 	detections,out_scores = get_gt(frame,frame_id,gt_dict)

# 	if detections is None:
# 		print("No dets")
# 		frame_id+=1
# 		continue

# 	detections = np.array(detections)
# 	out_scores = np.array(out_scores)
# 	print(len(detections) == 0)
# 	tracker,detections_class = deepsort.run_deep_sort(frame,out_scores,detections)

# 	for track in tracker.tracks:
# 		if not track.is_confirmed() or track.time_since_update > 1:
# 			continue

# 		bbox = track.to_tlbr() #Get the corrected/predicted bounding box
# 		id_num = str(track.track_id) #Get the ID for the particular track.
# 		features = track.features #Get the feature vector corresponding to the detection.

# 		pos_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

# 		# Read class for the frame in the current file from classes file
# 		obj_class = classes[pos_frame] if pos_frame in classes else -1
# 		# Read anomaly type, anomalous frame numbers from annotations file for the current file
# 		anomaly_in_frame = anomaly_info[2] if pos_frame in range(anomaly_info[0], (anomaly_info[1] + 1)) else 5
# 		input = [pos_frame, track.track_id, bbox[0], bbox[1], bbox[2], bbox[3], obj_class, anomaly_in_frame]	# How to add class and anomaly?
# 		inputs.append(input)

# 	if cv2.waitKey(1) & 0xFF == ord('q'):
# 		break

# 	frame_id+=1

# csv_file_name = '/content/drive/MyDrive/anomaly_detection/Inputs/input_{}.csv'.format(vid_name)
# fields = ['frame_num', 'track_id', 'x_min', 'y_min', 'x_max', 'y_max', 'object_class', 'anomaly_in_frame']

# with open(csv_file_name, 'w') as csvfile:
#     # creating a csv writer object
#     csvwriter = csv.writer(csvfile)

#     # writing the fields
#     csvwriter.writerow(fields)

#     # writing the data rows
#     csvwriter.writerows(inputs)

In [ ]:
# # l = ['Abuse005_x264.mp4',
# #   'Arrest003_x264.mp4',
# #   'Assault002_x264.mp4',
# #   'Burglary001_x264.mp4',
# #   'Burglary002_x264.mp4',
# #   'Burglary003_x264.mp4',
# #   'Fighting011_x264.mp4',
# #   'Normal_Videos_015_x264.mp4',
# #   'Normal_Videos_050_x264.mp4',
# #   'Normal_Videos_100_x264.mp4',
# #   'Normal_Videos_907_x264.mp4',
# #   'Normal_Videos_908_x264.mp4',
# #   'Normal_Videos_909_x264.mp4',
# #   'Normal_Videos_910_x264.mp4',
# #   'Normal_Videos_911_x264.mp4',
# #   'Normal_Videos_912_x264.mp4',
# #   'Normal_Videos_913_x264.mp4',
# #   'Normal_Videos_914_x264.mp4',
# #   'Normal_Videos_915_x264.mp4',
# #   'Normal_Videos_923_x264.mp4',
# #   'Normal_Videos_924_x264.mp4',
# #   'Normal_Videos_925_x264.mp4',
# #   'Normal_Videos_926_x264.mp4',
# #   'Normal_Videos_927_x264.mp4',
# #   'Normal_Videos_928_x264.mp4']
# for f in l:
#   print(f, ":", extract_anomaly_type(f))

In [ ]:
# import os

# l = os.listdir("/content/Test_dataset_2_unzipped/Test_dataset_2")
# done = os.listdir("/content/drive/MyDrive/anomaly_detection/Test")
# l = sorted(l)
# l2 = [i for i in l if "Normal" not in i and "test_"+i+".csv" not in done and i != ".DS_Store"]
# l2

In [ ]:
# import os
# c = 0
# l = []
# # for i in sorted(os.listdir("/content/drive/MyDrive/anomaly_detection/Inputs")):
# #   if 'Normal' in i:
# #     # print(i)
# #     l.append(i)
# #     c += 1
# # # print(len(l))
# # l
# # c
# (sorted(os.listdir("/content/drive/MyDrive/anomaly_detection/Test")))